In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pickle
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
def pg_hook():
    return psycopg2.connect(dbname="mydb", user="myuser", password="mypassword", host='localhost', port=5432)
conn = pg_hook()
conn.autocommit = True

In [2]:
q_pg = """
SELECT * FROM mlm.scores;
"""
df = pd.read_sql(q_pg, conn)
df.set_index('puid', drop=True, inplace=True)

In [3]:
q_sbs = """
SELECT puid, firstnames_source, firstnames_target FROM exploration.simplesbs;
"""
dfsbs = pd.read_sql(q_sbs, conn).set_index('puid', drop=True)

In [4]:
from fuzzywuzzy.fuzz import token_set_ratio
Xfwz = pd.DataFrame(index=dfsbs.index)
Xfwz['firstnames_tokenset'] = dfsbs[['firstnames_source', 'firstnames_target']].apply(lambda r: token_set_ratio(*r)/100, axis=1)

In [5]:
X= df[['firstnames_tfidf', 'surname_tfidf', 'locality_tfidf',
       'postalcodelong_tfidf', 'route_tfidf', 'title_tfidf',
       'firstnames_levenshtein', 'surname_levenshtein', 'route_levenshtein',
       'locality_levenshtein', 'postalcodelong_levenshtein',
       'firstnames_trg_metaphone', 'surname_trg_metaphone',
       'locality_trg_metaphone', 'route_trg_metaphone',
       'phone_exact', 'email_exact', 'ni_number_exact', 'geo_score_ths']].fillna(0)

In [6]:
X['firstnames_tokenset'] = Xfwz.loc[X.index, 'firstnames_tokenset']

## Load model

In [7]:
with open('/Users/pogier/Documents/63-Elk/dockersetup/shared_volume/ml_models/gradient_boosting.pickle' ,'rb') as f:
    model = pickle.load(f)

In [8]:
y_true = pd.read_sql(""" SELECT puid, y_true FROM mlm.ytrue;""", conn).set_index('puid')

In [9]:
ix_common = y_true.index.intersection(X.index)

In [10]:
model.score( X.loc[ix_common], y_true.loc[ix_common])

0.9937888198757764

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import  recall_score, accuracy_score, precision_score

In [12]:
y_pred = model.predict(X.loc[ix_common])
recall_score(y_true.loc[ix_common], y_pred)

0.9947698744769874

In [13]:
precision_score(y_true.loc[ix_common], y_pred)

0.9947698744769874

## Go to Predict

In [14]:
y_proba = pd.Series(data=model.predict_proba(X)[:,1], index=X.index, name='y_proba')


In [15]:
fp_export = '/Users/pogier/Documents/63-Elk/dockersetup/shared_volume/staging/y_proba.csv'

In [16]:
pd.DataFrame(y_proba).to_csv(fp_export, sep=',', encoding='utf-8', index=True)